First, import needed modules

In [39]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point, LineString, Polygon
import shapely.speedups
from geopy.geocoders import Nominatim
import os

Initalize variables

In [40]:
path = os.getcwd()
path

'/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters'

Load Datasets which have the endpoints of power line projects for 2009 and 2013

In [41]:
powerlines_13 = pd.read_csv(f'{path}/data/geodata/powerlines_2013.csv', sep=';')
powerlines_13.head()

,#BBPlG,Start,End,New,Retro,Int_start,Int_end,Short
0,1,Emden-Borssum,"52.20189, 7.03448",1.0,NaN,NaN,1.0,NaN
1,1,"52.20189, 7.03448",Osterath,1.0,NaN,1.0,NaN,NaN
2,2,Osterath,Philippsburg,1.0,NaN,NaN,NaN,NaN
3,3,Brunsbüttel,Aub,1.0,NaN,NaN,1.0,NaN
4,3,Aub,Großgartach,1.0,NaN,1.0,NaN,NaN


In [42]:
powerlines_09 = pd.read_csv(f'{path}/data/geodata/powerlines_2009.csv', sep=';')
powerlines_09.head()

,#ENLAG,Start,End,New,Retro,Int_start,Int_end,Short
0,1,Ellund,Hamburg Nord,1,NaN,NaN,NaN,NaN
1,1,Hamburg Nord,Dollern Stade,1,NaN,NaN,NaN,NaN
2,2,Ganderkesee,Wehrendorf Bad Essen,1,NaN,NaN,NaN,NaN
3,3,Neuenhagen bei Berlin,Bertikow,1,NaN,NaN,NaN,NaN
4,3,Bertikow,Gartz Oder,1,NaN,NaN,NaN,NaN


Note: 
- Int_Start and Int_End indicate whether a point was added to keep the direct line between points in Germany's boundaries
- Short indicates whether lines are < 20 km --> They are later dropped from the analysis

Get the geocoordinates of the 'endpoint' cities using OSM/Nominatim API

In [43]:
start_13 = geocode(powerlines_13['Start'], provider='nominatim', user_agent='plm', timeout=4)
start_13

,geometry,address
0,POINT (7.22429 53.34982),"Emden-Borssum, Birkenweg, Borssum, Emden, Nied..."
1,POINT (7.03585 52.20109),"Harberskamp, Eßseite, Riekenhof, Gronau, Kreis..."
2,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh..."
3,POINT (9.13954 53.89725),"Brunsbüttel, Dithmarschen, Schleswig-Holstein,..."
4,POINT (10.06529 49.55250),"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü..."
...,...,...
59,POINT (9.07063 54.08988),"Meldorf, Mitteldithmarschen, Dithmarschen, Sch..."
60,POINT (7.20583 53.36705),"Emden, Niedersachsen, Deutschland"
61,POINT (8.64012 48.89101),"Höhenstraße, Birkenfeld, Enzkreis, Baden-Württ..."
62,POINT (10.20084 47.92341),"Woringen, Bad Grönenbach, Landkreis Unterallgä..."


In [44]:
start_09 = geocode(powerlines_09['Start'], provider='nominatim', user_agent='plm', timeout=4)
start_09

,geometry,address
0,POINT (9.31077 54.79460),"Ellund, Handewitt, Schleswig-Flensburg, Schles..."
1,POINT (10.00974 53.61916),"Hamburg-Nord, Hamburg, Deutschland"
2,POINT (8.54515 53.03450),"Ganderkesee, Landkreis Oldenburg, Niedersachse..."
3,POINT (13.68893 52.52619),"Neuenhagen bei Berlin, Märkisch-Oderland, Bran..."
4,POINT (13.93302 53.24422),"Bertikow, Uckerfelde, Gramzow, Uckermark, Bran..."
5,POINT (11.86795 51.38743),"Goethestadt Bad Lauchstädt, Saalekreis, Sachse..."
6,POINT (7.25528 52.98163),"Dörpen West, 14, Deichstraße, Kleines Feld, He..."
7,POINT (10.36344 52.27176),"Wahle, Vechelde, Landkreis Peine, Niedersachse..."
8,POINT (10.40400 53.41391),"Krümmel, Geesthacht, Herzogtum Lauenburg, Schl..."
9,POINT (13.68893 52.52619),"Neuenhagen bei Berlin, Märkisch-Oderland, Bran..."


In [45]:
end_13 = geocode(powerlines_13['End'], provider='nominatim', user_agent='plm', timeout=4)
end_13

,geometry,address
0,POINT (7.03585 52.20109),"Harberskamp, Eßseite, Riekenhof, Gronau, Kreis..."
1,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh..."
2,POINT (8.45467 49.23675),"Philippsburg, Verwaltungsverband Philippsburg,..."
3,POINT (10.06529 49.55250),"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü..."
4,POINT (9.12587 49.14254),"Großgartach, Leingarten, Landkreis Heilbronn, ..."
...,...,...
59,POINT (8.85870 54.12985),"Büsum, Büsum-Wesselburen, Dithmarschen, Schles..."
60,POINT (8.05776 53.32577),"Conneforde, Wiefelstede, Landkreis Ammerland, ..."
61,POINT (8.84929 48.94873),"Mühlacker, VVG der Stadt Mühlacker, Enzkreis, ..."
62,POINT (10.18132 47.98677),"Memmingen, Bayern, 87700, Deutschland"


In [46]:
end_09 = geocode(powerlines_09['End'], provider='nominatim', user_agent='plm', timeout=4)
end_09

,geometry,address
0,POINT (10.00974 53.61916),"Hamburg-Nord, Hamburg, Deutschland"
1,POINT (9.54650 53.53788),"Dollern, Samtgemeinde Horneburg, Stade, Nieder..."
2,POINT (8.31328 52.33223),"Wehrendorf, Bad Essen, Landkreis Osnabrück, Ni..."
3,POINT (13.93302 53.24422),"Bertikow, Uckerfelde, Gramzow, Uckermark, Bran..."
4,POINT (14.39144 53.20858),"Gartz (Oder), Uckermark, Brandenburg, 16307, D..."
5,POINT (11.20783 50.17355),"Redwitz a.d.Rodach, Redwitz an der Rodach (VGe..."
6,POINT (6.61709 51.65769),"Wesel, Kreis Wesel, Nordrhein-Westfalen, Deuts..."
7,POINT (9.76125 50.91746),"Mecklar, Ludwigsau, Landkreis Hersfeld-Rotenbu..."
8,POINT (11.41480 53.62883),"Schwerin, Mecklenburg-Vorpommern, Deutschland"
9,POINT (13.27724 52.70254),"Birkenwerder Nord, Birkenwerder, Oberhavel, Br..."


Insert year column

In [47]:
dfs = [start_09, end_09, start_13, end_13]
years = [2009, 2009, 2013, 2013]
for df, year in zip(dfs, years):
    df['year'] = year

Create shapefiles from the start and endpoints dfs

In [48]:
# Start 09
pl_geo_start_09 = start_09.join(powerlines_09, how = 'left')
pl_geo_start_09 = pl_geo_start_09[['#ENLAG', 'Start', 'geometry', 'New', 'Retro', 'Int_start']]
pl_geo_start_09 = pl_geo_start_09.to_crs(epsg=25832)
pl_geo_start_09.to_file(f'{path}/data/geodata/startpoints_09.shp')
pl_geo_start_09.head()

,#ENLAG,Start,geometry,New,Retro,Int_start
0,1,Ellund,POINT (519981.129 6071979.603),1,NaN,NaN
1,1,Hamburg Nord,POINT (566789.619 5941623.917),1,NaN,NaN
2,2,Ganderkesee,POINT (469499.065 5876204.936),1,NaN,NaN
3,3,Neuenhagen bei Berlin,POINT (818017.258 5829902.470),1,NaN,NaN
4,3,Bertikow,POINT (829070.441 5910800.010),1,NaN,NaN


In [49]:
# Start 13
pl_geo_start_13 = start_13.join(powerlines_13, how = 'left')
pl_geo_start_13 = pl_geo_start_13[['#BBPlG', 'Start', 'geometry', 'New', 'Retro', 'Int_start']]
pl_geo_start_13 = pl_geo_start_13.to_crs(epsg=25832)
pl_geo_start_13.to_file(f'{path}/data/geodata/startpoints_13.shp')
pl_geo_start_13.head()

,#BBPlG,Start,geometry,New,Retro,Int_start
0,1,Emden-Borssum,POINT (381802.135 5912655.617),1.0,NaN,NaN
1,1,"52.20189, 7.03448",POINT (365771.558 5785222.130),1.0,NaN,1.0
2,2,Osterath,POINT (334024.420 5682438.683),1.0,NaN,NaN
3,3,Brunsbüttel,POINT (509169.440 5972099.313),1.0,NaN,NaN
4,3,Aub,POINT (577051.542 5489422.549),1.0,NaN,1.0


In [50]:
# End 09
pl_geo_end_09 = end_09.join(powerlines_09, how = 'left')
pl_geo_end_09 = pl_geo_end_09[['#ENLAG', 'End', 'geometry', 'New', 'Retro', 'Int_end']]
pl_geo_end_09 = pl_geo_end_09.to_crs(epsg=25832) #convert CRS to match BKG data
pl_geo_end_09.to_file(f'{path}/data/geodata/endpoints_09.shp')
pl_geo_end_09.head()

,#ENLAG,End,geometry,New,Retro,Int_end
0,1,Hamburg Nord,POINT (566789.619 5941623.917),1,NaN,NaN
1,1,Dollern Stade,POINT (536218.100 5932247.324),1,NaN,NaN
2,2,Wehrendorf Bad Essen,POINT (453206.201 5798213.062),1,NaN,NaN
3,3,Bertikow,POINT (829070.441 5910800.010),1,NaN,NaN
4,3,Gartz Oder,POINT (859924.674 5909051.896),1,NaN,NaN


In [51]:
# End 13
pl_geo_end_13 = end_13.join(powerlines_13, how = 'left')
pl_geo_end_13 = pl_geo_end_13[['#BBPlG', 'End', 'geometry', 'New', 'Retro', 'Int_end']]
pl_geo_end_13 = pl_geo_end_13.to_crs(epsg=25832) #convert CRS to match BKG data
pl_geo_end_13.to_file(f'{path}/data/geodata/endpoints_13.shp')
pl_geo_end_13.head()

,#BBPlG,End,geometry,New,Retro,Int_end
0,1,"52.20189, 7.03448",POINT (365771.558 5785222.130),1.0,NaN,1.0
1,1,Osterath,POINT (334024.420 5682438.683),1.0,NaN,NaN
2,2,Philippsburg,POINT (460302.514 5453917.732),1.0,NaN,NaN
3,3,Aub,POINT (577051.542 5489422.549),1.0,NaN,1.0
4,3,Großgartach,POINT (509180.155 5443309.167),1.0,NaN,NaN


Create powerlines_geo dataseta by merging start and endpoints with the powerlines dfs

In [52]:
# 09
pl_geo_09 = start_09.join(powerlines_09, how = 'left')
pl_geo_09.rename(columns = {'geometry':'Start_geometry', 'address':'Start_address'}, inplace = True)
pl_geo_09.drop('year', axis=1, inplace=True)
pl_geo_09 = end_09.join(pl_geo_09, how = 'left')
pl_geo_09.rename(columns = {'geometry':'End_geometry', 'address':'End_address'}, inplace = True)
pl_geo_09.head()

,End_geometry,End_address,year,Start_geometry,Start_address,#ENLAG,Start,End,New,Retro,Int_start,Int_end,Short
0,POINT (10.00974 53.61916),"Hamburg-Nord, Hamburg, Deutschland",2009,POINT (9.31077 54.79460),"Ellund, Handewitt, Schleswig-Flensburg, Schles...",1,Ellund,Hamburg Nord,1,NaN,NaN,NaN,NaN
1,POINT (9.54650 53.53788),"Dollern, Samtgemeinde Horneburg, Stade, Nieder...",2009,POINT (10.00974 53.61916),"Hamburg-Nord, Hamburg, Deutschland",1,Hamburg Nord,Dollern Stade,1,NaN,NaN,NaN,NaN
2,POINT (8.31328 52.33223),"Wehrendorf, Bad Essen, Landkreis Osnabrück, Ni...",2009,POINT (8.54515 53.03450),"Ganderkesee, Landkreis Oldenburg, Niedersachse...",2,Ganderkesee,Wehrendorf Bad Essen,1,NaN,NaN,NaN,NaN
3,POINT (13.93302 53.24422),"Bertikow, Uckerfelde, Gramzow, Uckermark, Bran...",2009,POINT (13.68893 52.52619),"Neuenhagen bei Berlin, Märkisch-Oderland, Bran...",3,Neuenhagen bei Berlin,Bertikow,1,NaN,NaN,NaN,NaN
4,POINT (14.39144 53.20858),"Gartz (Oder), Uckermark, Brandenburg, 16307, D...",2009,POINT (13.93302 53.24422),"Bertikow, Uckerfelde, Gramzow, Uckermark, Bran...",3,Bertikow,Gartz Oder,1,NaN,NaN,NaN,NaN


In [53]:
# 13
pl_geo_13 = start_13.join(powerlines_13, how = 'left')
pl_geo_13.rename(columns = {'geometry':'Start_geometry', 'address':'Start_address'}, inplace = True)
pl_geo_13.drop('year', axis=1, inplace=True)
pl_geo_13 = end_13.join(pl_geo_13, how = 'left')
pl_geo_13.rename(columns = {'geometry':'End_geometry', 'address':'End_address'}, inplace = True)
pl_geo_13.head()

,End_geometry,End_address,year,Start_geometry,Start_address,#BBPlG,Start,End,New,Retro,Int_start,Int_end,Short
0,POINT (7.03585 52.20109),"Harberskamp, Eßseite, Riekenhof, Gronau, Kreis...",2013,POINT (7.22429 53.34982),"Emden-Borssum, Birkenweg, Borssum, Emden, Nied...",1,Emden-Borssum,"52.20189, 7.03448",1.0,NaN,NaN,1.0,NaN
1,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",2013,POINT (7.03585 52.20109),"Harberskamp, Eßseite, Riekenhof, Gronau, Kreis...",1,"52.20189, 7.03448",Osterath,1.0,NaN,1.0,NaN,NaN
2,POINT (8.45467 49.23675),"Philippsburg, Verwaltungsverband Philippsburg,...",2013,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",2,Osterath,Philippsburg,1.0,NaN,NaN,NaN,NaN
3,POINT (10.06529 49.55250),"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü...",2013,POINT (9.13954 53.89725),"Brunsbüttel, Dithmarschen, Schleswig-Holstein,...",3,Brunsbüttel,Aub,1.0,NaN,NaN,1.0,NaN
4,POINT (9.12587 49.14254),"Großgartach, Leingarten, Landkreis Heilbronn, ...",2013,POINT (10.06529 49.55250),"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü...",3,Aub,Großgartach,1.0,NaN,1.0,NaN,NaN


Create direct line from start and end coordinates

In [54]:
dfs = [pl_geo_09, pl_geo_13]
for df in dfs:
    try:
        df.insert(loc=0, column='direct_line', value=0)
    except Exception:
        pass
    for i in range (0, len(df)):
        start = df['Start_geometry'].loc[df.index[i]] # get the start
        end = df['End_geometry'].loc[df.index[i]] # get the end
        df.loc[df.index[i], 'direct_line'] = LineString([start, end]) # create line

/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/pandas/core/internals/blocks.py:932: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return self.coerce_to_target_dtype(value).setitem(indexer, value)
/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/pandas/core/internals/managers.py:304: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/pandas/core/internals/blocks.py:932: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return self.coerce_to_target_dtype(value).setitem(indexer, value)
/opt/homebrew/Caskroom/minicond

In [55]:
pl_geo_09.head()

,direct_line,End_geometry,End_address,year,Start_geometry,Start_address,#ENLAG,Start,End,New,Retro,Int_start,Int_end,Short
0,"LINESTRING (9.3107737 54.794602, 10.0097398999...",POINT (10.00974 53.61916),"Hamburg-Nord, Hamburg, Deutschland",2009,POINT (9.31077 54.79460),"Ellund, Handewitt, Schleswig-Flensburg, Schles...",1,Ellund,Hamburg Nord,1,NaN,NaN,NaN,NaN
1,"LINESTRING (10.009739899920163 53.61915555, 9....",POINT (9.54650 53.53788),"Dollern, Samtgemeinde Horneburg, Stade, Nieder...",2009,POINT (10.00974 53.61916),"Hamburg-Nord, Hamburg, Deutschland",1,Hamburg Nord,Dollern Stade,1,NaN,NaN,NaN,NaN
2,"LINESTRING (8.5451469 53.0344984, 8.3132774 52...",POINT (8.31328 52.33223),"Wehrendorf, Bad Essen, Landkreis Osnabrück, Ni...",2009,POINT (8.54515 53.03450),"Ganderkesee, Landkreis Oldenburg, Niedersachse...",2,Ganderkesee,Wehrendorf Bad Essen,1,NaN,NaN,NaN,NaN
3,"LINESTRING (13.6889259 52.5261884, 13.93302354...",POINT (13.93302 53.24422),"Bertikow, Uckerfelde, Gramzow, Uckermark, Bran...",2009,POINT (13.68893 52.52619),"Neuenhagen bei Berlin, Märkisch-Oderland, Bran...",3,Neuenhagen bei Berlin,Bertikow,1,NaN,NaN,NaN,NaN
4,"LINESTRING (13.933023545115766 53.2442165, 14....",POINT (14.39144 53.20858),"Gartz (Oder), Uckermark, Brandenburg, 16307, D...",2009,POINT (13.93302 53.24422),"Bertikow, Uckerfelde, Gramzow, Uckermark, Bran...",3,Bertikow,Gartz Oder,1,NaN,NaN,NaN,NaN


Export shapefile

In [56]:
pl_geo_09.rename(columns = {'direct_line':'geometry'}, inplace = True)
pl_geo_09 = pl_geo_09.set_crs(epsg=4326) #set crs first
pl_geo_09 = pl_geo_09.to_crs(epsg=25832) # then convert to match BKG data
pl_geo_09 = pl_geo_09.reindex(columns=['#ENLAG', 'Start', 'End', 'geometry', 'New', 'Retro', 'Short'])
pl_geo_09.to_file(f'{path}/data/geodata/powerlines_09.shp')
pl_geo_09.head()

,#ENLAG,Start,End,geometry,New,Retro,Short
0,1,Ellund,Hamburg Nord,"LINESTRING (519981.129 6071979.603, 566789.619...",1,NaN,NaN
1,1,Hamburg Nord,Dollern Stade,"LINESTRING (566789.619 5941623.917, 536218.100...",1,NaN,NaN
2,2,Ganderkesee,Wehrendorf Bad Essen,"LINESTRING (469499.065 5876204.936, 453206.201...",1,NaN,NaN
3,3,Neuenhagen bei Berlin,Bertikow,"LINESTRING (818017.258 5829902.470, 829070.441...",1,NaN,NaN
4,3,Bertikow,Gartz Oder,"LINESTRING (829070.441 5910800.010, 859924.674...",1,NaN,NaN


In [57]:
dfs = [pl_geo_09, pl_geo_13]
pl_geo_13.rename(columns = {'direct_line':'geometry'}, inplace = True)
pl_geo_13 = pl_geo_13.set_crs(epsg=4326) #set crs first
pl_geo_13 = pl_geo_13.to_crs(epsg=25832) # then convert to match BKG data
pl_geo_13 = pl_geo_13.reindex(columns=['#BBPlG', 'Start', 'End', 'geometry', 'New', 'Retro', 'Short'])
pl_geo_13.to_file(f'{path}/data/geodata/powerlines_13.shp')
pl_geo_13.head()

,#BBPlG,Start,End,geometry,New,Retro,Short
0,1,Emden-Borssum,"52.20189, 7.03448","LINESTRING (381802.135 5912655.617, 365771.558...",1.0,NaN,NaN
1,1,"52.20189, 7.03448",Osterath,"LINESTRING (365771.558 5785222.130, 334024.420...",1.0,NaN,NaN
2,2,Osterath,Philippsburg,"LINESTRING (334024.420 5682438.683, 460302.514...",1.0,NaN,NaN
3,3,Brunsbüttel,Aub,"LINESTRING (509169.440 5972099.313, 577051.542...",1.0,NaN,NaN
4,3,Aub,Großgartach,"LINESTRING (577051.542 5489422.549, 509180.155...",1.0,NaN,NaN
